In [137]:
%matplotlib inline


Data Processing




In [ ]:
!pip install spacy

In [ ]:
!pip uninstall -y torchtext torch


Found existing installation: torchtext 0.18.0
Uninstalling torchtext-0.18.0:
  Successfully uninstalled torchtext-0.18.0
Found existing installation: torch 2.6.0
Uninstalling torch-2.6.0:
  Successfully uninstalled torch-2.6.0


In [ ]:
!pip install torchtext --no-cache-dir --upgrade --force-reinstall torch==2.3.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 201.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 120.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.2/779.2 MB 324.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 219.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 221.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 286.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 368.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 204.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 217.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 274.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 220.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━

In [139]:
import torch
print(torch.version)

<module 'torch.version' from '/usr/local/lib/python3.11/dist-packages/torch/version.py'>


In [138]:
import torchtext



In [ ]:
!python -m spacy download de_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 74.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
!python -m spacy download ru_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 83.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [140]:
import torch
import torch.nn as nn
import math
from torchtext.utils import download_from_url, extract_archive
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader
from collections import Counter
import io
import os

Data Loader

In [141]:
# -----------------------------
# Download and Load Multi30k
# -----------------------------
url_base = 'https://raw.githubusercontent.com/multi30k/dataset/master/data/task1/raw/'
train_urls = ('train.de.gz', 'train.en.gz')
val_urls = ('val.de.gz', 'val.en.gz')
test_urls = ('test_2016_flickr.de.gz', 'test_2016_flickr.en.gz')

train_filepaths = [extract_archive(download_from_url(url_base + url))[0] for url in train_urls]

In [142]:
# -----------------------------
# Tokenizers & Vocab
# -----------------------------
de_tokenizer = get_tokenizer('spacy', language='de_core_news_sm')
en_tokenizer = get_tokenizer('spacy', language='en_core_web_sm')

def build_vocab(filepath, tokenizer):
    counter = Counter()
    with io.open(filepath, encoding="utf8") as f:
        for line in f:
            counter.update(tokenizer(line))
    return build_vocab_from_iterator([tokenizer(line) for line in io.open(filepath, encoding="utf8")],
                                     specials=['<pad>', '<bos>', '<eos>', '<unk>'])

de_vocab = build_vocab(train_filepaths[0], de_tokenizer)
en_vocab = build_vocab(train_filepaths[1], en_tokenizer)

de_vocab.set_default_index(de_vocab['<unk>'])
en_vocab.set_default_index(en_vocab['<unk>'])

In [143]:
# -----------------------------
# Positional Encoding
# -----------------------------
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)  # [1, max_len, d_model]
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1)]
        return x


In [144]:
# -----------------------------
# Transformer Model
# -----------------------------
class TransformerTranslationModel(nn.Module):
    def __init__(self, src_vocab_size, trg_vocab_size, d_model=512, nhead=8,
                 num_encoder_layers=3, num_decoder_layers=3, dim_feedforward=2048, dropout=0.1):
        super().__init__()

        self.src_tok_emb = nn.Embedding(src_vocab_size, d_model)
        self.trg_tok_emb = nn.Embedding(trg_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model)

        self.transformer = nn.Transformer(d_model=d_model, nhead=nhead,
                                          num_encoder_layers=num_encoder_layers,
                                          num_decoder_layers=num_decoder_layers,
                                          dim_feedforward=dim_feedforward,
                                          dropout=dropout,
                                          batch_first=True)

        self.fc_out = nn.Linear(d_model, trg_vocab_size)
        self.d_model = d_model

    def forward(self, src, trg, src_mask=None, tgt_mask=None, src_padding_mask=None, tgt_padding_mask=None):
        src_emb = self.positional_encoding(self.src_tok_emb(src) * math.sqrt(self.d_model))
        tgt_emb = self.positional_encoding(self.trg_tok_emb(trg) * math.sqrt(self.d_model))

        output = self.transformer(src_emb, tgt_emb,
                                  src_mask=src_mask,
                                  tgt_mask=tgt_mask,
                                  src_key_padding_mask=src_padding_mask,
                                  tgt_key_padding_mask=tgt_padding_mask)

        return self.fc_out(output)


In [145]:
# -----------------------------
# Helper Functions
# -----------------------------
def generate_square_subsequent_mask(sz):
    return torch.triu(torch.full((sz, sz), float('-inf')), diagonal=1)

def encode_sentence(vocab, tokenizer, sentence):
    return [vocab['<bos>']] + [vocab[token] for token in tokenizer(sentence)] + [vocab['<eos>']]

def collate_fn(batch):
    src_batch, trg_batch = [], []
    for src_sample, trg_sample in batch:
        src_tensor = torch.tensor(encode_sentence(de_vocab, de_tokenizer, src_sample), dtype=torch.long)
        trg_tensor = torch.tensor(encode_sentence(en_vocab, en_tokenizer, trg_sample), dtype=torch.long)
        src_batch.append(src_tensor)
        trg_batch.append(trg_tensor)
    src_batch = nn.utils.rnn.pad_sequence(src_batch, padding_value=PAD_IDX, batch_first=True)
    trg_batch = nn.utils.rnn.pad_sequence(trg_batch, padding_value=PAD_IDX, batch_first=True)
    return src_batch, trg_batch

def load_data(path_de, path_en):
    with open(path_de, encoding="utf8") as f_de, open(path_en, encoding="utf8") as f_en:
        return list(zip(f_de.readlines(), f_en.readlines()))

def translate(model, sentence, src_vocab, trg_vocab, src_tokenizer, device, max_len=50):
    model.eval()

    # Encode German input sentence
    tokens = encode_sentence(src_vocab, src_tokenizer, sentence)
    src_tensor = torch.LongTensor(tokens).unsqueeze(0).to(device)  # shape: [1, src_len]

    # Start with <bos> token in target
    tgt_tokens = [trg_vocab['<bos>']]

    for _ in range(max_len):
        tgt_tensor = torch.LongTensor(tgt_tokens).unsqueeze(0).to(device)  # shape: [1, tgt_len]
        tgt_mask = generate_square_subsequent_mask(tgt_tensor.size(1)).to(device)

        with torch.no_grad():
            output = model(src_tensor, tgt_tensor, tgt_mask=tgt_mask)  # shape: [1, tgt_len, vocab_size]

        next_token = output[0, -1].argmax(-1).item()
        tgt_tokens.append(next_token)

        if next_token == trg_vocab['<eos>']:
            break

    # Convert token IDs to words (skip <bos> and <eos>)
    return " ".join([trg_vocab.get_itos()[i] for i in tgt_tokens[1:-1]])



In [146]:
# -----------------------------
# Load Dataset + Init Model
# -----------------------------
train_data = load_data(train_filepaths[0], train_filepaths[1])
train_loader = DataLoader(train_data, batch_size=32, shuffle=True, collate_fn=collate_fn)

model = TransformerTranslationModel(len(de_vocab), len(en_vocab))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

In [ ]:
# -----------------------------
# Training Loop
# -----------------------------
model.train()
for epoch in range(15):
    total_loss = 0
    for src, trg in train_loader:
        src, trg = src.to(device), trg.to(device)

        trg_input = trg[:, :-1]
        trg_output = trg[:, 1:].contiguous().view(-1)

        tgt_mask = generate_square_subsequent_mask(trg_input.size(1)).to(device)

        optimizer.zero_grad()
        output = model(src, trg_input, tgt_mask=tgt_mask)
        output = output.view(-1, output.shape[-1])

        loss = criterion(output, trg_output)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1} | Avg Loss: {avg_loss:.4f} | Perplexity: {math.exp(avg_loss):.2f}")

print("Training complete.")

Epoch 1 | Avg Loss: 1.5686 | Perplexity: 4.80
Epoch 2 | Avg Loss: 1.4168 | Perplexity: 4.12
Epoch 3 | Avg Loss: 1.2728 | Perplexity: 3.57
Epoch 4 | Avg Loss: 1.1384 | Perplexity: 3.12
Epoch 5 | Avg Loss: 1.0153 | Perplexity: 2.76
Epoch 6 | Avg Loss: 0.9010 | Perplexity: 2.46
Epoch 7 | Avg Loss: 0.7955 | Perplexity: 2.22
Epoch 8 | Avg Loss: 0.6978 | Perplexity: 2.01
Epoch 9 | Avg Loss: 0.6172 | Perplexity: 1.85
Epoch 10 | Avg Loss: 0.5427 | Perplexity: 1.72
Epoch 11 | Avg Loss: 0.4802 | Perplexity: 1.62
Epoch 12 | Avg Loss: 0.4308 | Perplexity: 1.54
Epoch 13 | Avg Loss: 0.3856 | Perplexity: 1.47
Epoch 14 | Avg Loss: 0.3480 | Perplexity: 1.42
Epoch 15 | Avg Loss: 0.3160 | Perplexity: 1.37
Training complete.


In [ ]:
torch.save(model.state_dict(), "transformer_de_en.pth")

In [147]:
model.load_state_dict(torch.load("transformer_de_en.pth", map_location=device))
model.eval()

TransformerTranslationModel(
  (src_tok_emb): Embedding(19215, 512)
  (trg_tok_emb): Embedding(10838, 512)
  (positional_encoding): PositionalEncoding()
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0-2): 3 x TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
          )
          (linear1): Linear(in_features=512, out_features=2048, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=2048, out_features=512, bias=True)
          (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
      )
      (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    )
    (decod

In [148]:
# Example translation
translated = translate(model, "ein mann steht neben einem auto", de_vocab, en_vocab, de_tokenizer, device)
print("Translated:", translated)

Translated: There is a couple standing next to a picnic table . 



In [149]:
# Convert to smaller
model.half()
torch.save(model.state_dict(), "transformer_de_en_fp16.pth")

In [150]:
model.load_state_dict(torch.load("transformer_de_en_fp16.pth", map_location=device))

<All keys matched successfully>

In [151]:
# Example translation
translated = translate(model, "ein mann steht neben einem auto", de_vocab, en_vocab, de_tokenizer, device)
print("Translated:", translated)

Translated: There is a couple standing next to a picnic table . 

